<a href="https://colab.research.google.com/github/k1lowM/sd-on-colabo/blob/main/controlnet_automatic1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 念のためコンテンツルートに移動
%cd /content

# webuiをインストール
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# sd-webui-controlnet のダウンロード
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet

# sd_dreambooth_extension のダウンロード
!git clone https://github.com/d8ahazard/sd_dreambooth_extension.git /content/stable-diffusion-webui/extensions/sd_dreambooth_extension

# 作業ディレクトリを移動させておく
%cd /content/stable-diffusion-webui

# モデルダウンロード関数の定義
!apt-get -y install -qq aria2 # aria2を使うと爆速で、モデルをダウンロードできる
def download_model(download_model_url, output_dir):
  # urlの最後がモデル名として、ファイル名を取得する
  file_name_path = download_model_url.split('/')[-1]
  # ダウンロードの実行
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_model_url} -d {output_dir} -o {file_name_path}
  # ダウンロード後の配置されたディレクトリを一応確認
  !ls -lh {output_dir}

modelsDir = "/content/stable-diffusion-webui/models/Stable-diffusion"

# stable-diffsion v1.4 のモデルのダウンロード
download_model("https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt", modelsDir)

# v1.4以外の学習済みモデルをダウンロード
learnedModel = True    # True = controlnetが動く，False = controlnetが動かない

if learnedModel:
  # stable-diffsion v2.1 model(768×768)のダウンロード
  #download_model("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors", modelsDir)
  #!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  # stable-diffsion v2.1 base model(512×512)のダウンロード
  download_model("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors", modelsDir)
  !wget "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml" -O {modelsDir}/v2-1_512-ema-pruned.yaml"

  # dreamlike-art
  download_model("https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors", modelsDir)

  # basil_mix
  download_model("https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors", modelsDir)
  download_model("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", "/content/stable-diffusion-webui/models/VAE")

  # Realism Engine 1.0
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/20414?type=Model&format=SafeTensor" -d {modelsDir} -o "realismEngine_v10.safetensors"
  !wget "https://civitai.com/api/download/models/20414?type=Config&format=Other" -O {modelsDir}/realismEngine_v10.yaml

  # nfixer & nrealfixer for Illuminati Diffusion v1.1
  !wget "https://civitai.com/api/download/models/15921" -O /content/stable-diffusion-webui/embeddings/nfixer.pt
  !wget "https://civitai.com/api/download/models/15927" -O /content/stable-diffusion-webui/embeddings/nrealfixer.pt
  
  # ChilloutMix
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/11745" -d {modelsDir} -o "chilloutmix_NiPrunedFp32Fix.safetensors"

# sd-webui-controlnetモデルをダウンロード
controlnet = True    # True = controlnetが動く，False = controlnetが動かない
controlnetHeavy = False
controlnetLight = True

controlnetModelsDir = "/content/stable-diffusion-webui/models/ControlNet"
if controlnet:
  # 重量版
  if controlnetHeavy:
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_hed.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_normal.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_seg.pth", controlnetModelsDir)

  # 軽量版をダウンロード
  if controlnetLight:
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors", controlnetModelsDir)

    # T2I-Adapter
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_canny-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_color-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_depth-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_openpose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_style-fp16.safetensors", controlnetModelsDir)

# webuiを別URLで立ち上げる
!python launch.py --share --xformers --enable-insecure-extension-access --ckpt {modelsDir}/sd-v1-4-full-ema.ckpt

/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 17160, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17160 (delta 11), reused 19 (delta 11), pack-reused 17133
Receiving objects: 100% (17160/17160), 27.91 MiB | 44.65 MiB/s, done.
Resolving deltas: 100% (11987/11987), done.
Cloning into '/content/stable-diffusion-webui/extensions/sd-webui-controlnet'...
remote: Enumerating objects: 3325, done.
remote: Total 3325 (delta 0), reused 0 (delta 0), pack-reused 3325
Receiving objects: 100% (3325/3325), 8.28 MiB | 25.85 MiB/s, done.
Resolving deltas: 100% (1734/1734), done.
Cloning into '/content/stable-diffusion-webui/extensions/sd_dreambooth_extension'...
remote: Enumerating objects: 6148, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 6148 (delta 17), reused 24 (delta 11), pack-reused 6108
Receiving objects: 100% (6148/6148), 1

In [ ]:
# GoogleドライブをGoogle Colab にマウント
# (Googleドライブ使わないなら実行の必要なし)

# from google.colab import drive
# drive.mount('/content/drive')